In [ ]:
from glob import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
def get_values_counts(raw_data_path):
    shapes = []
    for image_path in glob(raw_data_path, recursive=True):
        image = Image.open(image_path).convert('L')
        shapes.append(np.array(image).shape)
    
    values, counts = np.unique(shapes, return_counts=True, axis=0)

    return values, counts


def draw_random_shape(values, counts):
    probabilities = counts / counts.sum()
    index = np.random.choice(len(values), p=probabilities)
    random_shape = values[index]

    return random_shape


def normalize_pretrain_slice(slice):
    return cv2.normalize(slice, None, 0.0, 1.0, cv2.NORM_MINMAX, cv2.CV_32FC1)


def adjust_coordinates(x0, y0, h, w, max_h, max_w):
    x1 = x0 + h
    y1 = y0 + w
    
    if x1 > max_h:
        x0 = max_h - h
        x1 = max_h
        
    if y1 > max_w:
        y0 = max_w - w
        y1 = max_w
        
    return x0, x1, y0, y1


def get_slice_tiles_coords(values, counts, max_h=1259, max_w=300):
    tiles_coords = []
    
    for x0 in range(0, max_h, 126):
        for y0 in range(0, max_w, 100):
            h, w = draw_random_shape(values, counts)
            x0, x1, y0, y1 = adjust_coordinates(x0, y0, h, w, max_h, max_w)
            tiles_coords.append((x0, x1, y0, y1))
            
    return tiles_coords

In [ ]:
raw_data_path = "../data/raw/**/*.png"
values, counts = get_values_counts(raw_data_path)

In [ ]:
weighted_mean_height = np.average(values[:, 0], weights=counts)
weighted_mean_width = np.average(values[:, 1], weights=counts)

weighted_mean_height, weighted_mean_width

In [ ]:
tiles_coords = get_slice_tiles_coords(values, counts)

In [ ]:
slice_idx = 5
volume = np.load('../data/raw/pretrain/patch-the-planet-train-data-part5/seismicCubes_RFC_fullstack_2023.76358893.npy')
slice = volume[slice_idx, :, :].T

In [ ]:
volume

In [ ]:
for x0, x1, y0, y1 in tiles_coords:
    tile = slice[x0:x1, y0:y1]
    tile = normalize_pretrain_slice(tile)
    plt.imshow(tile, cmap='gray')
    plt.show()